## Local Content

In [ ]:
import boto3
import json
from IPython.display import Image
from PIL import Image, ImageDraw

client = boto3.client('rekognition')

pathA = ''
pathB = ''

# Open both files
with open(pathA, 'rb') as imageA, open(pathB, 'rb') as imageB:
    response = client.compare_faces(
        SourceImage={'Bytes': imageA.read()},
        TargetImage={'Bytes': imageB.read()},
        SimilarityThreshold=90,
#       QualityFilter='NONE'|'AUTO'|'LOW'|'MEDIUM'|'HIGH',
    )
    
    # If there is a match
    if len(response['FaceMatches']) > 0:
        for faceMatch in response['FaceMatches']:
            position = faceMatch['Face']['BoundingBox']
            similarity = str(faceMatch['Similarity'])
        
        # Show confidence
        print('Confidence: %.4f' % float(similarity) + '%')
        
        imageA = Image.open(imageA)
        imageB = Image.open(imageB)
        image_width, image_height = imageB.size 

        x1 = position['Left'] * image_width
        y1 = position['Top'] * image_height
        x2 = x1 + position['Width'] * image_width
        y2 = y1 + position['Height'] * image_height
        image_crop = imageB.crop((x1, y1, x2, y2))

        # Show matches
        draw = ImageDraw.Draw(imageB)
        points = (
                    (x1, y1),
                    (x2, y1),
                    (x2, y2),
                    (x1, y2),
                    (x1, y1)
                )
        draw.line(points, fill='#00d400', width=2)
        
        imageA.thumbnail((300, 300))
        imageB.thumbnail((300, 300))
        
        display(imageA, imageB)
        
        # Show RAW Data
        print('RAW Data:')
        print(json.dumps(response, indent=4))
        
    else:
        print('No Matches')

## Online Content

In [ ]:
import boto3
import json
import requests
from IPython.display import Image
from PIL import Image, ImageDraw
import io

client = boto3.client('rekognition')

urlA = ''
urlB = ''

respondA = requests.get(urlA).content
respondB = requests.get(urlB).content

response = client.compare_faces(
    SourceImage={'Bytes': respondA},
    TargetImage={'Bytes': respondB},
    SimilarityThreshold=90,
    #QualityFilter='NONE'|'AUTO'|'LOW'|'MEDIUM'|'HIGH',
)

if len(response['FaceMatches']) > 0:
    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']
        similarity = str(faceMatch['Similarity'])
        
        print('Confidence: %.4f' % float(similarity) + '%')
        
        imageA = Image.open(io.BytesIO(requests.get(urlA).content))
        imageB = Image.open(io.BytesIO(requests.get(urlB).content))
        image_width, image_height = imageB.size 

        x1 = position['Left'] * image_width
        y1 = position['Top'] * image_height
        x2 = x1 + position['Width'] * image_width
        y2 = y1 + position['Height'] * image_height
        image_crop = imageB.crop((x1, y1, x2, y2))
    
        draw = ImageDraw.Draw(imageB)
        points = (
                    (x1, y1),
                    (x2, y1),
                    (x2, y2),
                    (x1, y2),
                    (x1, y1)
                )
        draw.line(points, fill='#00d400', width=2)

        imageA.thumbnail((300, 300))
        imageB.thumbnail((300, 300))
        
        display(imageA, imageB)
    
        print('RAW Data:')
        print(json.dumps(response, indent=4))
        
else:
    print('No Matches')